In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import datetime
import pickle
import yaml
import pandas as pd

# Load annotation data

In [3]:
annotation_data = pd.read_csv('./data/annotations.csv', low_memory=False)

In [4]:
list(enumerate(annotation_data.columns))

[(0, 'split'),
 (1, 'sensor_id'),
 (2, 'audio_filename'),
 (3, 'annotator_id'),
 (4, 'borough'),
 (5, 'block'),
 (6, 'latitude'),
 (7, 'longitude'),
 (8, 'year'),
 (9, 'week'),
 (10, 'day'),
 (11, 'hour'),
 (12, '1-1_small-sounding-engine_presence'),
 (13, '1-2_medium-sounding-engine_presence'),
 (14, '1-3_large-sounding-engine_presence'),
 (15, '1-X_engine-of-uncertain-size_presence'),
 (16, '2-1_rock-drill_presence'),
 (17, '2-2_jackhammer_presence'),
 (18, '2-3_hoe-ram_presence'),
 (19, '2-4_pile-driver_presence'),
 (20, '2-X_other-unknown-impact-machinery_presence'),
 (21, '3-1_non-machinery-impact_presence'),
 (22, '4-1_chainsaw_presence'),
 (23, '4-2_small-medium-rotating-saw_presence'),
 (24, '4-3_large-rotating-saw_presence'),
 (25, '4-X_other-unknown-powered-saw_presence'),
 (26, '5-1_car-horn_presence'),
 (27, '5-2_car-alarm_presence'),
 (28, '5-3_siren_presence'),
 (29, '5-4_reverse-beeper_presence'),
 (30, '5-X_other-unknown-alert-signal_presence'),
 (31, '6-1_stationary-mu

In [5]:
annotation_data.iloc[:, :12].sample(2).T

,23469,19883
split,train,train
sensor_id,28,27
audio_filename,28_012004.wav,27_000315.wav
annotator_id,4468,109
borough,1,1
block,1315,547
latitude,40.7501,40.7303
longitude,-73.974,-73.9956
year,2018,2017
week,4,29


In [6]:
annotation_data['dt'] = annotation_data.apply(
    lambda x: f"{x.year}-{x.week}-{x.day + 1}",
    axis=1
).map(lambda x: datetime.datetime.strptime(x, '%Y-%W-%u'))

In [7]:
annotation_data['date_slno'] = (annotation_data.dt - annotation_data.dt.min()).dt.days

In [8]:
annotation_data = annotation_data.sort_values('audio_filename')
file_list = annotation_data['audio_filename'].unique().tolist()
len(file_list)

18515

# Train/validate/test dataset exploration

In [9]:
annotation_data.shape

(57449, 80)

In [10]:
annotation_data.audio_filename.drop_duplicates().shape

(annotation_data
 .loc[:, ['split', 'audio_filename']]
 .drop_duplicates()
).shape

(annotation_data
 .loc[:, ['split', 'audio_filename', 'borough', 'block', 'latitude', 'longitude', 'year', 'week', 'day', 'hour', 'dt']]
 .drop_duplicates()
).shape

(18515,)

(18515, 2)

(18515, 11)

In [11]:
tmp = (annotation_data
       .loc[:, ['split', 'audio_filename', 'borough', 'block', 'latitude', 'longitude', 'year', 'week', 'day', 'hour', 'dt']]
       .drop_duplicates())

In [12]:
tmp.loc[:, ['split', 'audio_filename']].split.value_counts()

train       13538
validate     4308
test          669
Name: split, dtype: int64

In [13]:
tmp.loc[lambda x: x.split == 'train'].borough.drop_duplicates().tolist()
tmp.loc[lambda x: x.split == 'validate'].borough.drop_duplicates().tolist()
tmp.loc[lambda x: x.split == 'test'].borough.drop_duplicates().tolist()

[1, 3, 4]

[1, 3]

[1, 3, 4]

In [14]:
train_blocks = set(tmp.loc[lambda x: x.split == 'train'].block.tolist())
validate_blocks = set(tmp.loc[lambda x: x.split == 'validate'].block.tolist())
test_blocks = set(tmp.loc[lambda x: x.split == 'test'].block.tolist())

len(validate_blocks - train_blocks)
len(test_blocks - train_blocks)

4

13

In [15]:
tmp.loc[lambda x: x.split == 'train'].dt.min(), tmp.loc[lambda x: x.split == 'train'].dt.max()
tmp.loc[lambda x: x.split == 'validate'].dt.min(), tmp.loc[lambda x: x.split == 'validate'].dt.max()
tmp.loc[lambda x: x.split == 'test'].dt.min(), tmp.loc[lambda x: x.split == 'test'].dt.max()

(Timestamp('2016-05-21 00:00:00'), Timestamp('2019-04-18 00:00:00'))

(Timestamp('2016-05-07 00:00:00'), Timestamp('2019-04-18 00:00:00'))

(Timestamp('2019-04-22 00:00:00'), Timestamp('2020-01-05 00:00:00'))

In [16]:
tmp.loc[lambda x: x.split == 'train'].latitude.min(), tmp.loc[lambda x: x.split == 'train'].latitude.max()
tmp.loc[lambda x: x.split == 'validate'].latitude.min(), tmp.loc[lambda x: x.split == 'validate'].latitude.max()
tmp.loc[lambda x: x.split == 'test'].latitude.min(), tmp.loc[lambda x: x.split == 'test'].latitude.max()

(40.69413, 40.77991)

(40.693979999999996, 40.73642)

(40.67517, 40.77991)

In [17]:
tmp.loc[lambda x: x.split == 'train'].longitude.min(), tmp.loc[lambda x: x.split == 'train'].longitude.max()
tmp.loc[lambda x: x.split == 'validate'].longitude.min(), tmp.loc[lambda x: x.split == 'validate'].longitude.max()
tmp.loc[lambda x: x.split == 'test'].longitude.min(), tmp.loc[lambda x: x.split == 'test'].longitude.max()

(-74.00499, -73.85184)

(-73.99728, -73.98697)

(-74.01576, -73.85184)

# Parse taxonomy

In [18]:
with open('./data/dcase-ust-taxonomy.yaml', 'r') as f:
    taxonomy = yaml.load(f, Loader=yaml.Loader)

In [19]:
full_fine_target_labels = ["{}-{}_{}".format(coarse_id, fine_id, fine_label)
                           for coarse_id, fine_dict in taxonomy['fine'].items()
                           for fine_id, fine_label in fine_dict.items()]

fine_target_labels = [x
                      for x in full_fine_target_labels
                      if x.split('_')[0].split('-')[1] != 'X']

coarse_target_labels = ["_".join([str(k), v])
                        for k, v in taxonomy['coarse'].items()]

In [20]:
full_fine_target_labels
fine_target_labels
coarse_target_labels

['1-1_small-sounding-engine',
 '1-2_medium-sounding-engine',
 '1-3_large-sounding-engine',
 '1-X_engine-of-uncertain-size',
 '2-1_rock-drill',
 '2-2_jackhammer',
 '2-3_hoe-ram',
 '2-4_pile-driver',
 '2-X_other-unknown-impact-machinery',
 '3-1_non-machinery-impact',
 '4-1_chainsaw',
 '4-2_small-medium-rotating-saw',
 '4-3_large-rotating-saw',
 '4-X_other-unknown-powered-saw',
 '5-1_car-horn',
 '5-2_car-alarm',
 '5-3_siren',
 '5-4_reverse-beeper',
 '5-X_other-unknown-alert-signal',
 '6-1_stationary-music',
 '6-2_mobile-music',
 '6-3_ice-cream-truck',
 '6-X_music-from-uncertain-source',
 '7-1_person-or-small-group-talking',
 '7-2_person-or-small-group-shouting',
 '7-3_large-crowd',
 '7-4_amplified-speech',
 '7-X_other-unknown-human-voice',
 '8-1_dog-barking-whining']

['1-1_small-sounding-engine',
 '1-2_medium-sounding-engine',
 '1-3_large-sounding-engine',
 '2-1_rock-drill',
 '2-2_jackhammer',
 '2-3_hoe-ram',
 '2-4_pile-driver',
 '3-1_non-machinery-impact',
 '4-1_chainsaw',
 '4-2_small-medium-rotating-saw',
 '4-3_large-rotating-saw',
 '5-1_car-horn',
 '5-2_car-alarm',
 '5-3_siren',
 '5-4_reverse-beeper',
 '6-1_stationary-music',
 '6-2_mobile-music',
 '6-3_ice-cream-truck',
 '7-1_person-or-small-group-talking',
 '7-2_person-or-small-group-shouting',
 '7-3_large-crowd',
 '7-4_amplified-speech',
 '8-1_dog-barking-whining']

['1_engine',
 '2_machinery-impact',
 '3_non-machinery-impact',
 '4_powered-saw',
 '5_alert-signal',
 '6_music',
 '7_human-voice',
 '8_dog']

In [21]:
tmp1 = pd.DataFrame.from_records([
    (key1, key2, val2)
    for key1, val1 in taxonomy['fine'].items()
    for key2, val2 in val1.items()
], columns=['coarse_id', 'fine_id', 'fine'])
tmp2 = pd.DataFrame.from_records([
    (key, val)
    for key, val in taxonomy['coarse'].items()
], columns=['coarse_id', 'coarse'])
taxonomy_df = pd.merge(tmp1, tmp2, on='coarse_id', how='inner')
taxonomy_df = taxonomy_df.astype('str')
taxonomy_df['coarse'] = taxonomy_df.coarse_id + '_' + taxonomy_df.coarse
taxonomy_df['fine'] = taxonomy_df.coarse_id + '-' + taxonomy_df.fine_id + '_' + taxonomy_df.fine

In [22]:
taxonomy_df

,coarse_id,fine_id,fine,coarse
0,1,1,1-1_small-sounding-engine,1_engine
1,1,2,1-2_medium-sounding-engine,1_engine
2,1,3,1-3_large-sounding-engine,1_engine
3,1,X,1-X_engine-of-uncertain-size,1_engine
4,2,1,2-1_rock-drill,2_machinery-impact
5,2,2,2-2_jackhammer,2_machinery-impact
6,2,3,2-3_hoe-ram,2_machinery-impact
7,2,4,2-4_pile-driver,2_machinery-impact
8,2,X,2-X_other-unknown-impact-machinery,2_machinery-impact
9,3,1,3-1_non-machinery-impact,3_non-machinery-impact


# Separate train and valid data, only zooniverse platform

In [23]:
train_data = (annotation_data.copy()
              .loc[lambda x: x.split == 'train']
              .loc[lambda x: x.annotator_id > 0]
              .set_index('audio_filename')
              .filter(regex='.*_presence$', axis='columns')
              .rename(columns=lambda x: x[:-9])
              .astype('int64'))

In [24]:
(train_data.reset_index()
 .assign(cnt=1).groupby('audio_filename')['cnt'].sum()
 .value_counts()
)

3    13538
Name: cnt, dtype: int64

In [25]:
coarse_train = train_data.loc[:, sorted(taxonomy_df.coarse.unique())]
fine_train = train_data.loc[:, sorted(taxonomy_df.fine.tolist())]

In [26]:
coarse_train.shape, fine_train.shape, len(set(coarse_train.index.tolist()))

((40614, 8), (40614, 29), 13538)

---

In [27]:
valid_data = (annotation_data.copy()
              .loc[lambda x: x.split == 'validate']
              .loc[lambda x: x.annotator_id > 0]
              .set_index('audio_filename')
              .filter(regex='.*_presence$', axis='columns')
              .rename(columns=lambda x: x[:-9])
              .astype('int64'))

In [28]:
(valid_data.reset_index()
 .assign(cnt=1).groupby('audio_filename')['cnt'].sum()
 .value_counts()
)

3    4308
Name: cnt, dtype: int64

In [29]:
coarse_valid = valid_data.loc[:, sorted(taxonomy_df.coarse.unique())]
fine_valid = valid_data.loc[:, sorted(taxonomy_df.fine.tolist())]

In [30]:
coarse_valid.shape, fine_valid.shape, len(set(fine_valid.index.tolist()))

((12924, 8), (12924, 29), 4308)

In [31]:
coarse_train.sort_index(inplace=True)
coarse_valid.sort_index(inplace=True)
fine_train.sort_index(inplace=True)
fine_valid.sort_index(inplace=True)

In [32]:
coarse_train.head(2)
coarse_valid.head(2)
fine_train.head(2)
fine_valid.head(2)

,1_engine,2_machinery-impact,3_non-machinery-impact,4_powered-saw,5_alert-signal,6_music,7_human-voice,8_dog
audio_filename,,,,,,,,
01_000006.wav,0,0,0,0,0,0,0,0
01_000006.wav,0,0,0,0,0,0,0,1


,1_engine,2_machinery-impact,3_non-machinery-impact,4_powered-saw,5_alert-signal,6_music,7_human-voice,8_dog
audio_filename,,,,,,,,
00_000066.wav,1,0,0,0,0,0,0,0
00_000066.wav,1,0,0,0,0,0,0,0


,1-1_small-sounding-engine,1-2_medium-sounding-engine,1-3_large-sounding-engine,1-X_engine-of-uncertain-size,2-1_rock-drill,2-2_jackhammer,2-3_hoe-ram,2-4_pile-driver,2-X_other-unknown-impact-machinery,3-1_non-machinery-impact,...,6-1_stationary-music,6-2_mobile-music,6-3_ice-cream-truck,6-X_music-from-uncertain-source,7-1_person-or-small-group-talking,7-2_person-or-small-group-shouting,7-3_large-crowd,7-4_amplified-speech,7-X_other-unknown-human-voice,8-1_dog-barking-whining
audio_filename,,,,,,,,,,,,,,,,,,,,,
01_000006.wav,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01_000006.wav,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


,1-1_small-sounding-engine,1-2_medium-sounding-engine,1-3_large-sounding-engine,1-X_engine-of-uncertain-size,2-1_rock-drill,2-2_jackhammer,2-3_hoe-ram,2-4_pile-driver,2-X_other-unknown-impact-machinery,3-1_non-machinery-impact,...,6-1_stationary-music,6-2_mobile-music,6-3_ice-cream-truck,6-X_music-from-uncertain-source,7-1_person-or-small-group-talking,7-2_person-or-small-group-shouting,7-3_large-crowd,7-4_amplified-speech,7-X_other-unknown-human-voice,8-1_dog-barking-whining
audio_filename,,,,,,,,,,,,,,,,,,,,,
00_000066.wav,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00_000066.wav,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---

In [33]:
metadata_cols = [
 'sensor_id',
 'audio_filename',
 'borough',
 'block',
 'latitude',
 'longitude',
 'year',
 'week',
 'day',
 'hour',
 'dt',
 'date_slno']

In [34]:
train_metadata = (annotation_data.copy()
                  .loc[lambda x: x.split == 'train']
                  .loc[:, metadata_cols]
                  .drop_duplicates()
                  .set_index('audio_filename'))
valid_metadata = (annotation_data.copy()
                  .loc[lambda x: x.split == 'validate']
                  .loc[:, metadata_cols]
                  .drop_duplicates()
                  .set_index('audio_filename'))

In [35]:
train_metadata.shape, valid_metadata.shape

((13538, 11), (4308, 11))

In [36]:
train_metadata.head(2)
valid_metadata.head(2)

,sensor_id,borough,block,latitude,longitude,year,week,day,hour,dt,date_slno
audio_filename,,,,,,,,,,,
01_000006.wav,1,1,541,40.73033,-73.9987,2017,17,0,10,2017-04-24,352
01_000038.wav,1,1,541,40.73033,-73.9987,2017,21,2,17,2017-05-24,382


,sensor_id,borough,block,latitude,longitude,year,week,day,hour,dt,date_slno
audio_filename,,,,,,,,,,,
00_000066.wav,0,1,547,40.72951,-73.99388,2017,28,2,0,2017-07-12,431
00_000071.wav,0,1,547,40.72951,-73.99388,2016,50,5,13,2016-12-17,224


# Separate train and valid data, only ground truth

In [37]:
train_data_gt = (annotation_data.copy()
                 .loc[lambda x: x.split == 'train']
                 .loc[lambda x: x.annotator_id == 0]
                 .set_index('audio_filename')
                 .filter(regex='.*_presence$', axis='columns')
                 .rename(columns=lambda x: x[:-9])
                 .astype('int64'))

In [38]:
(train_data_gt.reset_index()
 .assign(cnt=1).groupby('audio_filename')['cnt'].sum()
 .value_counts()
)

1    178
Name: cnt, dtype: int64

In [39]:
coarse_train_gt = train_data_gt.loc[:, sorted(taxonomy_df.coarse.unique())]
fine_train_gt = train_data_gt.loc[:, sorted(taxonomy_df.fine.tolist())]

In [40]:
coarse_train_gt.shape, fine_train_gt.shape, len(set(coarse_train_gt.index.tolist()))

((178, 8), (178, 29), 178)

In [41]:
valid_data_gt = (annotation_data.copy()
                 .loc[lambda x: x.split == 'validate']
                 .loc[lambda x: x.annotator_id == 0]
                 .set_index('audio_filename')
                 .filter(regex='.*_presence$', axis='columns')
                 .rename(columns=lambda x: x[:-9])
                 .astype('int64'))

In [42]:
(valid_data_gt.reset_index()
 .assign(cnt=1).groupby('audio_filename')['cnt'].sum()
 .value_counts()
)

1    538
Name: cnt, dtype: int64

In [43]:
coarse_valid_gt = valid_data_gt.loc[:, sorted(taxonomy_df.coarse.unique())]
fine_valid_gt = valid_data_gt.loc[:, sorted(taxonomy_df.fine.tolist())]

In [44]:
coarse_valid_gt.shape, fine_valid_gt.shape, len(set(coarse_valid_gt.index.tolist()))

((538, 8), (538, 29), 538)

In [45]:
coarse_train_gt.sort_index(inplace=True)
coarse_valid_gt.sort_index(inplace=True)
fine_train_gt.sort_index(inplace=True)
fine_valid_gt.sort_index(inplace=True)

In [46]:
coarse_train_gt.head(2)
coarse_valid_gt.head(2)
fine_train_gt.head(2)
fine_valid_gt.head(2)

,1_engine,2_machinery-impact,3_non-machinery-impact,4_powered-saw,5_alert-signal,6_music,7_human-voice,8_dog
audio_filename,,,,,,,,
01_010234.wav,0,0,0,0,0,0,0,1
01_010270.wav,0,0,0,0,0,0,0,1


,1_engine,2_machinery-impact,3_non-machinery-impact,4_powered-saw,5_alert-signal,6_music,7_human-voice,8_dog
audio_filename,,,,,,,,
00_000066.wav,1,0,0,0,0,0,0,0
00_000118.wav,1,0,0,0,0,0,1,0


,1-1_small-sounding-engine,1-2_medium-sounding-engine,1-3_large-sounding-engine,1-X_engine-of-uncertain-size,2-1_rock-drill,2-2_jackhammer,2-3_hoe-ram,2-4_pile-driver,2-X_other-unknown-impact-machinery,3-1_non-machinery-impact,...,6-1_stationary-music,6-2_mobile-music,6-3_ice-cream-truck,6-X_music-from-uncertain-source,7-1_person-or-small-group-talking,7-2_person-or-small-group-shouting,7-3_large-crowd,7-4_amplified-speech,7-X_other-unknown-human-voice,8-1_dog-barking-whining
audio_filename,,,,,,,,,,,,,,,,,,,,,
01_010234.wav,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
01_010270.wav,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


,1-1_small-sounding-engine,1-2_medium-sounding-engine,1-3_large-sounding-engine,1-X_engine-of-uncertain-size,2-1_rock-drill,2-2_jackhammer,2-3_hoe-ram,2-4_pile-driver,2-X_other-unknown-impact-machinery,3-1_non-machinery-impact,...,6-1_stationary-music,6-2_mobile-music,6-3_ice-cream-truck,6-X_music-from-uncertain-source,7-1_person-or-small-group-talking,7-2_person-or-small-group-shouting,7-3_large-crowd,7-4_amplified-speech,7-X_other-unknown-human-voice,8-1_dog-barking-whining
audio_filename,,,,,,,,,,,,,,,,,,,,,
00_000066.wav,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00_000118.wav,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


---

In [47]:
train_metadata_gt = (annotation_data.copy()
                     .loc[lambda x: x.split == 'train']
                     .loc[lambda x: x.annotator_id == 0]
                     .loc[:, metadata_cols]
                     .drop_duplicates()
                     .set_index('audio_filename'))
valid_metadata_gt = (annotation_data.copy()
                     .loc[lambda x: x.split == 'validate']
                     .loc[lambda x: x.annotator_id == 0]
                     .loc[:, metadata_cols]
                     .drop_duplicates()
                     .set_index('audio_filename'))

In [48]:
train_metadata_gt.shape, valid_metadata_gt.shape

((178, 11), (538, 11))

In [49]:
train_metadata_gt.head(2)
valid_metadata_gt.head(2)

,sensor_id,borough,block,latitude,longitude,year,week,day,hour,dt,date_slno
audio_filename,,,,,,,,,,,
01_010234.wav,1,1,541,40.73033,-73.9987,2018,10,6,7,2018-03-11,673
01_010270.wav,1,1,541,40.73033,-73.9987,2018,21,2,7,2018-05-23,746


,sensor_id,borough,block,latitude,longitude,year,week,day,hour,dt,date_slno
audio_filename,,,,,,,,,,,
00_000066.wav,0,1,547,40.72951,-73.99388,2017,28,2,0,2017-07-12,431
00_000118.wav,0,1,547,40.72951,-73.99388,2017,40,0,21,2017-10-02,513


# Save metadata

In [51]:
with open('./data/metadata.pkl', 'wb') as f:
    pickle.dump({
        'coarse_train': coarse_train,
        'coarse_valid': coarse_valid,
        'fine_train': fine_train,
        'fine_valid': fine_valid,
        'train_metadata': train_metadata,
        'valid_metadata': valid_metadata,
        'coarse_train_gt': coarse_train_gt,
        'coarse_valid_gt': coarse_valid_gt,
        'fine_train_gt': fine_train_gt,
        'fine_valid_gt': fine_valid_gt,
        'train_metadata_gt': train_metadata_gt,
        'valid_metadata_gt': valid_metadata_gt,
        'taxonomy_df': taxonomy_df
    }, f)